SYSTEME DE RECOMMANDATION DE FILMS

Nous avons utilisé un dataset MovieLens 100k Small Dataset provenant de Kaggle contenant 2 fichiers: 
    - movies.csv contenant les informations sur les films(movieId, title et genre)
    - rating.csv contenant les évaluations des utilisateurs sur les films(userId, movieId, rating, timestamp)

Le fichier movies.csv contient 3 variables et 9125 observations; le fichier rating.csv a 4 variables dont 100004 observations.

1- IMPORT ET CHARGEMENT DES DONNEES

In [123]:
import pandas as pd

movies = pd.read_csv("data/movies.csv", sep=";")
ratings = pd.read_csv("data/rating.csv", sep=";")

movies.head()
# ratings.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


2- DATA CLEANINIG MINIMAL

a- Structure des données

In [124]:
movies.shape

(9125, 3)

In [125]:
ratings.shape

(100004, 4)

In [126]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [127]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


b- Vérification des valeurs manquantes

In [128]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [129]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

c- Suppression des doublons

In [130]:
movies = movies.drop_duplicates(subset=["movieId"])

In [131]:
ratings = ratings.drop_duplicates(subset=["userId", "movieId"])

d- Nettoyage de la colonne genres

In [136]:
movies['genres'] = movies['genres'].str.replace('|', ' ', regex=False)

e- Suppression des films sans genre

In [137]:
movies = movies[movies['genres'] != '(no genres listed)']

cette action est effectuée parce qu'un film sans genre est impossible à comparer aux autres.

In [138]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


3- Implémentation du coeur du système de recommandation

In [142]:
from sklearn.feature_extraction.text import TfidfVectorizer #librairie pour transformer les textes en nombres
from sklearn.metrics.pairwise import cosine_similarity #librairie pour calculer la similarité entre deux vecteurs(choses)

#on crée le transformeur TF-IDF qui sait comment transformer des mots en nombres, pour le moment il n'a encore rien appris
tfidf = TfidfVectorizer()

#on fait apprendre le transformeur, il va apprendre tous les genres de films(fit) puis les tranformer en nombres(transform) 
tfidf_matrix = tfidf.fit_transform(movies['genres'])

#on calcule la similarité entre les films
cosine_sim = cosine_similarity(tfidf_matrix)

#on crée une fonction qui prend en entrée le titre d'un film et qui retourne une liste de 10 films recommandés
def recommend_movies(movie_title, cosine_sim=cosine_sim):

    #on vérifie si le film existe
    if movie_title not in movies['title'].values:
        return "Film non trouvé"
    
    #on récupère l'index du film donné en entrée
    idx = movies[movies['title'] == movie_title].index[0]
    
    #on récupère les similarités de ce film avec tous les autres films
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #on trie les films en fonction de leur similarité avec le film donné en entrée
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #on prend les 10 films les plus similaires (en excluant le premier qui est le film lui-même)
    sim_scores = sim_scores[1:11]
    
    #on récupère les indices des films recommandés
    movie_indices = [i[0] for i in sim_scores]
    
    #on retourne les titres des films recommandés
    return movies.iloc[movie_indices][['title', 'genres']]

recommend_movies("Bed of Roses (1996)")

,title,genres
24,Leaving Las Vegas (1995),Drama Romance
27,Persuasion (1995),Drama Romance
33,Carrington (1995),Drama Romance
44,How to Make an American Quilt (1995),Drama Romance
47,When Night Is Falling (1995),Drama Romance
69,Bed of Roses (1996),Drama Romance
77,Once Upon a Time... When We Were Colored (1995),Drama Romance
79,Angels and Insects (1995),Drama Romance
97,"Bridges of Madison County, The (1995)",Drama Romance
117,Frankie Starlight (1995),Drama Romance
